In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta

In [ ]:
df = pd.read_csv("../files/students.csv", index_col = 0)

# Pair Programming ETL Transformación I

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

    1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

    2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:

    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea

- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [ ]:
# os recomendamos resetear el index del dataframe 
# de los datos climáticos para que no se repitan 
# los nombres de las columnas.


# El primer problema al que nos podemos enfrentar 
# es que si vemos los tipos de las columnas vemos 
# que estas columnas son objetos, es decir, strings,
#  lo que hará que trabajar con ellas sea un poco 
# complicado: 

clima.dtypes

timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object

# en Python tenemos la librería `ast` que nos 
# permite castear un string que dentro tiene 
# diccionarios, o listas o tuplas a su tipo 
# correspondiente. En nuestro caso, lo que 
# conseguiremos es no tener strings sino listas 
# en la columna. Esto lo haremos de 
# la siguiente forma: 

import ast

clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada,
#  una fantasía de Pandas es que si hago un
#  apply sobre una columna cuyos valores 
# son diccionarios o listas nos va a genererar
#  una columna con los valores de los diccionarios
#  o listas. Donde cada columna será key 
# del diccionario o cada elemento de la lista. 


x = clima['wind_profile'].apply(pd.Series)


# nos creamos un dataframe nuevo con el 
# resultado de la información de una de 
# las columnas separadas por columnas.
#  Esto nos va a devolver un dataframe donde 
# cada fila será una celda del dataframe 
# anterior. 

x = df['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores 
# tuvieramos en la lista. Donde columna es,
#  en este caso, un diccionario 
# (porque nuestra lista esta compuesta 
# por distintos diccionarios)

# Ok, hemos conseguido desempaquetar 
# la información de la lista en distintas 
# columnas. Ahora tenemos que despempaquetar
#  la información de los diccionarios en
#  distintas columnas. En este caso, 
# lo que querremos es que las key de 
# los diccionarios sean los nombres de 
# las columnas y los values los valores 
# de las celdas del dataframe. 
# Volveremos a seguir entonces la misma lógica
# que antes con el apply, pero en este caso 
# necesitamos hacerlo para todo el dataframe
#  (que es x): 

# Por eso empezamos con un for para iterar 
# por cada una de las columnas. 

for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore
    #  de la key "layer" y lo almacenamos en 
    # una variable que convertimos a string 

    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se 
    # llame valores para "guardar" los valores 
    # de la celda

    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes 
    # para ir añadiendo esta información a el 
    # dataframe con la información del clima. 

    df.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos 
# columnas ya podremos hacer el gropuby para 
# después unir toda la información. 

- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.